In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, classification_report, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb

In [2]:
df = pd.read_excel("data/0527_final.xlsx")

In [ ]:
# columns to use
df.columns == ['year', 'IGFC', 'gvkey', 'coname', 'cusip', 'co_per_rol', 'forced_3', 'departure_code', 'forced_3_4', 'tenure', 'I3y_lag', 'sic', 'ff17_code', 'execid', 
               'gender', 'page', 'becameceo', 'leftofc', 'roa', 'gross_margin', 'operating_margin', 'asset_turnover', 'receivables_turnover', 'inventory_turnover', 'debt_to_assets', 'equity_ratio', 'debt_to_equity', 'current_ratio', 'quick_ratio', 'working_capital', 'ocf_to_assets', 'ocf_to_sales', 
               'ebit_to_assets', 'ebitda_to_assets', 'pi_to_assets', 'ib_to_assets', 'txt_to_assets', 'xint_to_assets', 'capx_to_assets', 'dp_to_assets', 'emp_to_assets', 'tdc1_to_assets', 'bonus_to_assets', 'ppent_to_assets', 're_to_assets', 'rect_to_assets', 'salary_to_assets', 'option_to_asset', 
               'roa_3y_lag', 'gross_margin_3y_lag', 'operating_margin_3y_lag', 'asset_turnover_3y_lag', 'receivables_turnover_3y_lag', 'inventory_turnover_3y_lag', 'debt_to_assets_3y_lag', 'equity_ratio_3y_lag', 'debt_to_equity_3y_lag', 'current_ratio_3y_lag', 'quick_ratio_3y_lag', 'working_capital_3y_lag', 'ocf_to_assets_3y_lag', 'ocf_to_sales_3y_lag', 
               'ebit_to_assets_3y_lag', 'ebitda_to_assets_3y_lag', 'pi_to_assets_3y_lag', 'ib_to_assets_3y_lag', 'txt_to_assets_3y_lag', 'xint_to_assets_3y_lag', 'capx_to_assets_3y_lag', 'dp_to_assets_3y_lag', 'emp_to_assets_3y_lag', 'tdc1_to_assets_3y_lag', 'bonus_to_assets_3y_lag', 'ppent_to_assets_3y_lag', 're_to_assets_3y_lag', 'rect_to_assets_3y_lag', 'salary_to_assets_3y_lag', 'option_to_asset_3y_lag', 
               'Iff1', 'Iff2', 'Iff3', 'Iff4', 'Iff5', 'Iff6', 'Iff7', 'Iff8', 'Iff9', 'Iff10', 'Iff11', 'Iff12', 'Iff13', 'Iff14', 'Iff15', 'Iff16', 'Iff17', 
               'I1992', 'I1993', 'I1994', 'I1995', 'I1996', 'I1997', 'I1998', 'I1999', 'I2000', 'I2001', 'I2002', 'I2003', 'I2004', 'I2005', 'I2006', 'I2007', 'I2008', 'I2009', 'I2010', 'I2011', 'I2012', 'I2013', 'I2014', 'I2015', 'I2016', 'I2017', 'I2018', 'I2019', 'I2020', 'I2021', 'I2022']

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

# boosting

In [ ]:
import pandas as pd
import numpy as np # import numpy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


def var_select_boosting(
    df: pd.DataFrame,
    k: int,
    X: list,
    y: str,
    start: int = 1992,
    end: int = 2022,
    boosting: str = 'xgb',
    year_col: str = None,
    test_size: float = 0.2,
    random_state: int = 42
):
    # (Omitted sections identical to previous code)
    feature_cols = X
    target_col = y
    boosting_method = boosting.lower()

    # --- 1. Input Validation ---
    # (Same as before)
    if boosting_method not in ['ada', 'xgb', 'lgbm']:
        raise ValueError("boosting_method must be one of 'ada', 'xgb', or 'lgbm'.")
    if target_col not in df.columns:
        raise ValueError(f"Target column '{target_col}' not found in the DataFrame.")
    missing_features = [col for col in feature_cols if col not in df.columns]
    if missing_features:
        raise ValueError(f"Feature columns '{missing_features}' not found in the DataFrame.")
    if not isinstance(k, int) or k <= 0:
        raise ValueError("k must be a positive integer.")
    if k > len(feature_cols):
        warnings.warn(f"k ({k}) is larger than the number of available features ({len(feature_cols)}). Selecting all features.")
        k = len(feature_cols)

    # --- 2. Year Filtering (Optional) ---
    df_to_use = df.copy()
    if year_col:
        if year_col not in df_to_use.columns:
            raise ValueError(f"Year column '{year_col}' not found in the DataFrame.")
        print(f"Filtering data for years {start}-{end} using column '{year_col}'.")
        try:
            df_to_use[year_col] = pd.to_numeric(df_to_use[year_col], errors='raise')
        except ValueError as e:
            raise ValueError(f"Cannot convert year column '{year_col}' to numeric. Error: {e}")
        
        df_to_use = df_to_use[(df_to_use[year_col] >= start) & (df_to_use[year_col] <= end)]
        if df_to_use.empty:
            raise ValueError(f"No data found for the specified year range ({start}-{end}).")
    else:
        print("Note: 'year_col' not specified. Using the entire DataFrame.")

    if df_to_use.empty:
        raise ValueError("DataFrame is empty after filtering.")

    # --- 3. Data Preparation (X_data, y_data) and Conditional NaN/inf Handling ---
    X_data_original = df_to_use[feature_cols]
    y_processed = df_to_use[target_col].copy() # Assuming y is numeric and has no NaNs

    X_data = X_data_original.copy()
    
    print(f"Number of rows in X_data/y_processed before preparation: {len(X_data)}")

    # 1. Unify inf values to NaN in X_data for all boosting methods
    # Can use .to_numpy() for efficient check or DataFrame's replace method
    inf_present_in_X = np.isinf(X_data.select_dtypes(include=np.number).to_numpy()).any()
    if inf_present_in_X:
        print("Converting inf values in feature data (X_data) to NaN.")
        X_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    if boosting_method in ['ada']:
        print(f"When using '{boosting_method.upper()}', rows with NaN in feature data (X_data) will be removed (inf is handled after being converted to NaN).")
        # Now, NaNs in X_data are either original NaNs or NaNs converted from inf
        nan_in_X_mask = X_data.isnull().any(axis=1)
        
        if nan_in_X_mask.any(): 
            X_data_before_drop_len = len(X_data)
            X_data = X_data[~nan_in_X_mask]
            y_processed = y_processed[~nan_in_X_mask] # Filter y_processed accordingly

            rows_dropped_count = X_data_before_drop_len - len(X_data)
            if rows_dropped_count > 0:
                 print(f"Removed {rows_dropped_count} rows containing NaN (or former inf) in feature data (X_data).")
        else:
            if inf_present_in_X: # inf was present but after conversion to NaN, no other NaNs existed
                 print("inf in feature data (X_data) was converted to NaN, but no rows were removed as there were no additional NaNs.")
            else: # No NaN/inf to begin with
                 print("No rows were removed as there were no NaN/inf values in the feature data (X_data).")
        
        if X_data.empty: 
            raise ValueError("No data remains after removing NaN/inf rows from feature data (X_data).")
    else: # e.g., 'lgbm', 'xgb'
        print(f"When using '{boosting_method.upper()}', NaNs in feature data (X_data) (inf is converted to NaN) are handled internally, so rows are not removed.")
        if X_data.isnull().any().any(): # If NaNs actually exist (original or converted from inf)
            print("Note: Feature data (X_data) contains NaN values (some converted from former inf), but the model handles them internally.")
        elif inf_present_in_X: # Only inf was converted to NaN, and no other NaNs exist
             print("Note: inf values in feature data (X_data) have been converted to NaN. The model handles NaNs internally.")

    print(f"Final number of rows in X_data/y_processed: {len(X_data)}")

    # Assuming y_processed is numeric with no NaNs (user should verify)
    if not pd.api.types.is_numeric_dtype(y_processed):
        warnings.warn(f"Warning: Target variable '{target_col}' was assumed to be numeric, but its actual type is {y_processed.dtype}.")
    if y_processed.isnull().any():
         warnings.warn(f"Warning: Target variable '{target_col}' was assumed to have no NaNs, but NaN values were found.")

    # --- 4. Feature Type Check Based on Model ---
    # (Same as before)
    if boosting_method in ['ada']:
        non_numeric_cols = X_data.select_dtypes(exclude=np.number).columns
        if len(non_numeric_cols) > 0:
            raise ValueError(
                f"{boosting_method.upper()} model requires all feature variables to be numeric. "
                f"The following non-numeric columns were found: {list(non_numeric_cols)}. "
                "Please perform preprocessing such as Label Encoding or One-Hot Encoding."
            )
    elif boosting_method == 'lgbm':
        for col in X_data.select_dtypes(include='object').columns:
            print(f"For LightGBM, converting object type column '{col}' to 'category' type.")
            X_data.loc[:, col] = X_data[col].astype('category')

    # --- 5. Data Splitting (Train/Test) ---
    # (Same as before)
    if not isinstance(y_processed, pd.Series): # Check if y_processed is a Series
        y_processed = pd.Series(y_processed, name=target_col, index=X_data.index if len(y_processed) == len(X_data) else None)

    is_classification_target = y_processed.nunique() > 1 and y_processed.nunique() < len(y_processed) * 0.8
    stratify_option = y_processed if is_classification_target else None
    if stratify_option is not None:
        class_counts = y_processed.value_counts()
        if (class_counts < 2).any(): 
            warnings.warn("Stratified sampling is not possible because some classes in the target variable have only one sample. Proceeding with regular sampling.")
            stratify_option = None
            
    try:
        X_train, X_test, y_train_np, y_test_np = train_test_split(
            X_data, y_processed, test_size=test_size, random_state=random_state, stratify=stratify_option
        )
    except ValueError as e:
        warnings.warn(f"Error during data splitting (likely related to stratify): {e}. Retrying with regular splitting.")
        X_train, X_test, y_train_np, y_test_np = train_test_split(
            X_data, y_processed, test_size=test_size, random_state=random_state, stratify=None
        )

    y_train = pd.Series(y_train_np, name=target_col, index=X_train.index)
    y_test = pd.Series(y_test_np, name=target_col, index=X_test.index)
    
    # --- 6. Initialize and Train Boosting Model ---
    # (Same as before)
    model = None
    if boosting_method == 'ada':
        model = AdaBoostClassifier(random_state=random_state)
    elif boosting_method == 'xgb':
        xgb_params = {'random_state': random_state}
        model = XGBClassifier(**xgb_params)
    elif boosting_method == 'lgbm':
        model = LGBMClassifier(random_state=random_state, verbosity=-1)

    model.fit(X_train, y_train) 

    # --- 7. Extract Feature Importances ---
    # (Same as before)
    importances = model.feature_importances_

    # --- 8. Select Top K Features ---
    # (Same as before)
    feature_importance_df = pd.DataFrame({
        'feature': X_train.columns,
        'importance': importances
    })
    top_k_features_df = feature_importance_df.sort_values(by='importance', ascending=False).head(k)
    selected_feature_names = top_k_features_df['feature'].tolist()

    # --- 9. Reconstruct X Datasets with Selected Features ---
    # (Same as before)
    X_train_selected = X_train[selected_feature_names]
    X_test_selected = X_test[selected_feature_names]

    # --- 10. Visualize Feature Importances ---
    # (Same as before)
    fig, ax = plt.subplots(figsize=(12, max(6, k * 0.55)))
    sns.barplot(x='importance', y='feature', data=top_k_features_df, ax=ax, palette="plasma")
    ax.set_title(f'"{boosting_method.upper()}" model top {k} importance variables', fontsize=16)
    ax.set_xlabel('Importance', fontsize=14)
    ax.set_ylabel('Feature', fontsize=14)
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    plt.tight_layout()

    return top_k_features_df, fig

In [5]:
exec_cols = ['gender', 'page', 'tenure', 'salary_to_assets', 'emp_to_assets', 'tdc1_to_assets', 'bonus_to_assets','option_to_asset']
firm_cols = ['ebit_to_assets', 'ebitda_to_assets', 'pi_to_assets', 'ib_to_assets', 'txt_to_assets', 'xint_to_assets', 'capx_to_assets', 'dp_to_assets', 'emp_to_assets', 'tdc1_to_assets', 'bonus_to_assets', 'ppent_to_assets', 're_to_assets', 'rect_to_assets', 'salary_to_assets', 'option_to_asset']

In [ ]:
# using forced_3 as target variable
# result3_exec: A DataFrame storing important variables from exec_cols, sorted by importance.
# result3_firm: A DataFrame storing important variables from firm_cols, sorted by importance.
'''
result3_exec = pd.DataFrame()
result3_firm = pd.DataFrame()

top_k_features_df, fig = var_select_boosting(
    df = df, k = len(exec_cols), X = exec_cols, y = 'forced_3')
result3_exec['exec_all'] = top_k_features_df['feature'].tolist()

top_k_features_df, fig = var_select_boosting(
    df = df, k = len(firm_cols), X = firm_cols, y = 'forced_3')
result3_firm['firm_all'] = top_k_features_df['feature'].tolist()

for industry in df['ff17_code'].unique():
    print(f"Industry: {industry}")
    industry_df = df[df['ff17_code'] == industry]
    if len(industry_df) < 10:
        print(f"Not enough data for industry {industry} to perform variable selection.")
        continue
    colname_exec = f"exec_{industry}"
    colname_firm = f"firm_{industry}"
    top_k_features_df, fig = var_select_boosting(
        df = industry_df, k = len(exec_cols), X = exec_cols, y = 'forced_3')
    result3_exec[colname_exec] = top_k_features_df['feature'].tolist()
    top_k_features_df, fig = var_select_boosting(
        df = industry_df, k = len(firm_cols), X = firm_cols, y = 'forced_3')
    result3_firm[colname_firm] = top_k_features_df['feature'].tolist()
    #plt.show()  # Show the plot for each industry
    
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(exec_cols), end = 2008, X = exec_cols, y = 'forced_3', year_col='year')
result3_exec['exec_before'] = top_k_features_df['feature'].tolist()
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(firm_cols), end = 2008, X = firm_cols, y = 'forced_3', year_col='year')
result3_firm['firm_before'] = top_k_features_df['feature'].tolist()
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(exec_cols), start = 2009, X = exec_cols, y = 'forced_3', year_col='year')
result3_exec['exec_after'] = top_k_features_df['feature'].tolist()
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(firm_cols), start = 2009, X = firm_cols, y = 'forced_3', year_col='year')
result3_firm['firm_after'] = top_k_features_df['feature'].tolist()

result3_exec.to_csv("data/var_select_exec_3.csv", index=False)
result3_firm.to_csv("data/var_select_firm_3.csv", index=False)
'''

In [ ]:
# using forced_34 as target variable
# result34_exec: A DataFrame storing important variables from exec_cols, sorted by importance.
# result34_firm: A DataFrame storing important variables from firm_cols, sorted by importance.
'''
result34_exec = pd.DataFrame()
result34_firm = pd.DataFrame()

top_k_features_df, fig = var_select_boosting(
    df = df, k = len(exec_cols), X = exec_cols, y = 'forced_3_4')
result34_exec['exec_all'] = top_k_features_df['feature'].tolist()

top_k_features_df, fig = var_select_boosting(
    df = df, k = len(firm_cols), X = firm_cols, y = 'forced_3_4')
result34_firm['firm_all'] = top_k_features_df['feature'].tolist()

for industry in df['ff17_code'].unique():
    print(f"Industry: {industry}")
    industry_df = df[df['ff17_code'] == industry]
    if len(industry_df) < 10:
        print(f"Not enough data for industry {industry} to perform variable selection.")
        continue
    colname_exec = f"exec_{industry}"
    colname_firm = f"firm_{industry}"
    top_k_features_df, fig = var_select_boosting(
        df = industry_df, k = len(exec_cols), X = exec_cols, y = 'forced_3_4')
    result34_exec[colname_exec] = top_k_features_df['feature'].tolist()
    top_k_features_df, fig = var_select_boosting(
        df = industry_df, k = len(firm_cols), X = firm_cols, y = 'forced_3_4')
    result34_firm[colname_firm] = top_k_features_df['feature'].tolist()
    #plt.show()  # Show the plot for each industry
    
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(exec_cols), end = 2008, X = exec_cols, y = 'forced_3_4', year_col='year')
result34_exec['exec_before'] = top_k_features_df['feature'].tolist()
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(firm_cols), end = 2008, X = firm_cols, y = 'forced_3_4', year_col='year')
result34_firm['firm_before'] = top_k_features_df['feature'].tolist()
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(exec_cols), start = 2009, X = exec_cols, y = 'forced_3_4', year_col='year')
result34_exec['exec_after'] = top_k_features_df['feature'].tolist()
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(firm_cols), start = 2009, X = firm_cols, y = 'forced_3_4', year_col='year')
result34_firm['firm_after'] = top_k_features_df['feature'].tolist()

result34_exec.to_csv("data/var_select_exec_34.csv", index=False)
result34_firm.to_csv("data/var_select_firm_34.csv", index=False)
'''

In [ ]:
# using all variables as features
# result_all: A DataFrame storing important variables from all columns, sorted by importance.
# before_3: A DataFrame storing important variables before 2009(financial crisis), sorted by importance.
# after_3: A DataFrame storing important variables after 2009(financial crisis), sorted by importance.

'''
result_all = pd.DataFrame()

cols = ['gender', 'page', 'roa', 'gross_margin', 'operating_margin', 'asset_turnover', 'receivables_turnover', 'inventory_turnover', 'debt_to_assets', 'equity_ratio', 'debt_to_equity', 'current_ratio', 'quick_ratio', 'working_capital', 'ocf_to_assets', 'ocf_to_sales', 
        'ebit_to_assets', 'ebitda_to_assets', 'pi_to_assets', 'ib_to_assets', 'txt_to_assets', 'xint_to_assets', 'capx_to_assets', 'dp_to_assets', 'emp_to_assets', 'tdc1_to_assets', 'bonus_to_assets', 'ppent_to_assets', 're_to_assets', 'rect_to_assets', 'salary_to_assets', 'option_to_asset', 
        'roa_3y_lag', 'gross_margin_3y_lag', 'operating_margin_3y_lag', 'asset_turnover_3y_lag', 'receivables_turnover_3y_lag', 'inventory_turnover_3y_lag', 'debt_to_assets_3y_lag', 'equity_ratio_3y_lag', 'debt_to_equity_3y_lag', 'current_ratio_3y_lag', 'quick_ratio_3y_lag', 'working_capital_3y_lag', 'ocf_to_assets_3y_lag', 'ocf_to_sales_3y_lag', 
        'ebit_to_assets_3y_lag', 'ebitda_to_assets_3y_lag', 'pi_to_assets_3y_lag', 'ib_to_assets_3y_lag', 'txt_to_assets_3y_lag', 'xint_to_assets_3y_lag', 'capx_to_assets_3y_lag', 'dp_to_assets_3y_lag', 'emp_to_assets_3y_lag', 'tdc1_to_assets_3y_lag', 'bonus_to_assets_3y_lag', 'ppent_to_assets_3y_lag', 're_to_assets_3y_lag', 'rect_to_assets_3y_lag', 'salary_to_assets_3y_lag', 'option_to_asset_3y_lag']

top_k_features_df, fig = var_select_boosting(
    df = df, k = len(cols), X = cols, y = 'forced_3')
result_all['all_3'] = top_k_features_df['feature'].tolist()

for industry in df['ff17_code'].unique():
    print(f"Industry: {industry}")
    industry_df = df[df['ff17_code'] == industry]
    if len(industry_df) < 10:
        print(f"Not enough data for industry {industry} to perform variable selection.")
        continue
    colname = f"{industry}"
    top_k_features_df, fig = var_select_boosting(
        df = industry_df, k = len(cols), X = cols, y = 'forced_3')
    result_all[colname + '_3'] = top_k_features_df['feature'].tolist()
    #plt.show()  # Show the plot for each industry
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(cols), end = 2008, X = cols, y = 'forced_3', year_col='year')
result_all['all_before_3'] = top_k_features_df['feature'].tolist()
top_k_features_df, fig = var_select_boosting(
    df = df, k = len(cols), start = 2009, X = cols, y = 'forced_3', year_col='year')
result_all['all_after_3'] = top_k_features_df['feature'].tolist()


top_k_features_df, fig = var_select_boosting(
    df = df, k = len(cols), X = cols, y = 'forced_3_4')
result_all['all_34'] = top_k_features_df['feature'].tolist()

for industry in df['ff17_code'].unique():
    print(f"Industry: {industry}")
    industry_df = df[df['ff17_code'] == industry]
    if len(industry_df) < 10:
        print(f"Not enough data for industry {industry} to perform variable selection.")
        continue
    colname = f"{industry}"
    top_k_features_df, fig = var_select_boosting(
        df = industry_df, k = len(cols), X = cols, y = 'forced_3_4')
    result_all[colname + '_34'] = top_k_features_df['feature'].tolist()

top_k_features_df, fig = var_select_boosting(
    df = df, k = len(cols), end = 2008, X = cols, y = 'forced_3_4', year_col='year')
result_all['all_before_34'] = top_k_features_df['feature'].tolist()

top_k_features_df, fig = var_select_boosting(
    df = df, k = len(cols), start = 2009, X = cols, y = 'forced_3_4', year_col='year')
result_all['all_after_34'] = top_k_features_df['feature'].tolist()

result_all.to_csv("data/var_select_all.csv", index=False)
'''

In [23]:
result_all = pd.read_csv("data/var_select_all.csv")
result_all

,all_3,13_3,17_3,11_3,6_3,7_3,15_3,9_3,16_3,3_3,...,1_34,14_34,5_34,10_34,2_34,12_34,8_34,4_34,all_before_34,all_after_34
0,roa,bonus_to_assets_3y_lag,ib_to_assets,ib_to_assets,roa,roa,roa,ocf_to_assets_3y_lag,debt_to_assets,roa,...,inventory_turnover_3y_lag,roa,debt_to_assets_3y_lag,operating_margin,ib_to_assets,ebit_to_assets_3y_lag,ocf_to_assets_3y_lag,ebit_to_assets,pi_to_assets,ib_to_assets
1,pi_to_assets,pi_to_assets_3y_lag,rect_to_assets_3y_lag,gross_margin_3y_lag,quick_ratio_3y_lag,inventory_turnover_3y_lag,dp_to_assets_3y_lag,re_to_assets,bonus_to_assets,ppent_to_assets_3y_lag,...,roa,gross_margin_3y_lag,quick_ratio,xint_to_assets,debt_to_assets,ebit_to_assets,operating_margin_3y_lag,bonus_to_assets,ib_to_assets,option_to_asset_3y_lag
2,bonus_to_assets,ocf_to_assets,pi_to_assets,pi_to_assets_3y_lag,pi_to_assets_3y_lag,debt_to_assets_3y_lag,ebit_to_assets,quick_ratio_3y_lag,xint_to_assets,salary_to_assets_3y_lag,...,bonus_to_assets_3y_lag,ocf_to_sales_3y_lag,pi_to_assets_3y_lag,current_ratio,equity_ratio,asset_turnover_3y_lag,re_to_assets_3y_lag,txt_to_assets_3y_lag,bonus_to_assets,page
3,ib_to_assets,ocf_to_assets_3y_lag,option_to_asset,current_ratio_3y_lag,ebitda_to_assets,xint_to_assets_3y_lag,debt_to_assets_3y_lag,page,operating_margin,ocf_to_assets,...,ebit_to_assets_3y_lag,ebitda_to_assets,roa,salary_to_assets,debt_to_assets_3y_lag,quick_ratio_3y_lag,xint_to_assets_3y_lag,emp_to_assets,page,option_to_asset
4,option_to_asset,equity_ratio_3y_lag,bonus_to_assets,ebit_to_assets_3y_lag,ppent_to_assets_3y_lag,working_capital_3y_lag,pi_to_assets,equity_ratio_3y_lag,ocf_to_assets,ebit_to_assets,...,dp_to_assets_3y_lag,pi_to_assets_3y_lag,ocf_to_assets_3y_lag,pi_to_assets,roa,re_to_assets_3y_lag,roa,salary_to_assets,capx_to_assets_3y_lag,inventory_turnover_3y_lag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,xint_to_assets_3y_lag,emp_to_assets_3y_lag,ocf_to_sales_3y_lag,dp_to_assets,ib_to_assets_3y_lag,ocf_to_assets_3y_lag,ebitda_to_assets_3y_lag,operating_margin_3y_lag,ib_to_assets_3y_lag,operating_margin_3y_lag,...,equity_ratio_3y_lag,gender,ib_to_assets_3y_lag,emp_to_assets_3y_lag,ib_to_assets_3y_lag,debt_to_equity_3y_lag,asset_turnover_3y_lag,current_ratio_3y_lag,xint_to_assets,capx_to_assets
58,ocf_to_assets_3y_lag,gender,ebitda_to_assets_3y_lag,receivables_turnover,ebit_to_assets_3y_lag,pi_to_assets_3y_lag,ib_to_assets_3y_lag,debt_to_assets_3y_lag,ppent_to_assets_3y_lag,roa_3y_lag,...,receivables_turnover_3y_lag,quick_ratio_3y_lag,txt_to_assets_3y_lag,ppent_to_assets_3y_lag,ppent_to_assets_3y_lag,capx_to_assets_3y_lag,capx_to_assets_3y_lag,asset_turnover_3y_lag,txt_to_assets,equity_ratio_3y_lag
59,roa_3y_lag,roa_3y_lag,capx_to_assets,equity_ratio,txt_to_assets_3y_lag,ib_to_assets_3y_lag,pi_to_assets_3y_lag,inventory_turnover_3y_lag,bonus_to_assets_3y_lag,pi_to_assets_3y_lag,...,gender,operating_margin_3y_lag,ebit_to_assets_3y_lag,re_to_assets_3y_lag,tdc1_to_assets_3y_lag,xint_to_assets_3y_lag,txt_to_assets_3y_lag,ib_to_assets_3y_lag,ocf_to_sales_3y_lag,re_to_assets_3y_lag
60,xint_to_assets,ib_to_assets_3y_lag,current_ratio,roa_3y_lag,ebitda_to_assets_3y_lag,emp_to_assets_3y_lag,salary_to_assets_3y_lag,txt_to_assets_3y_lag,rect_to_assets_3y_lag,ib_to_assets_3y_lag,...,ib_to_assets_3y_lag,txt_to_assets_3y_lag,ebitda_to_assets_3y_lag,salary_to_assets_3y_lag,emp_to_assets_3y_lag,ebitda_to_assets_3y_lag,ebit_to_assets_3y_lag,ebitda_to_assets_3y_lag,ebitda_to_assets_3y_lag,asset_turnover_3y_lag


# tenure

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

def find_optimal_tenure_bins_xgb(df_input: pd.DataFrame,
                                           tenure_col: str = 'tenure',
                                           target_col: str = 'forced_3',
                                           max_tree_depth: int = 3,
                                           random_state: int = 42) -> list:
    """
    Suggests bins for a tenure column using an XGBoost model with a single tree.
    It extracts the split points for the tenure variable from the XGBoost tree structure.
    """
    df_processed = df_input.copy()

    # Remove rows with NaN in essential columns.
    df_processed.dropna(subset=[tenure_col, target_col], inplace=True)

    X = df_processed[[tenure_col]]
    y_original = df_processed[target_col]
    
    # Transform the target variable using LabelEncoder.
    le = LabelEncoder()
    y = le.fit_transform(y_original)

    # Initialize and train the XGBoost model
    model = xgb.XGBClassifier(
        n_estimators=1,             # Use only one tree
        max_depth=max_tree_depth,
        random_state=random_state,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    model.fit(X, y) # An error may occur here if X or y are unsuitable.

    # Extract split point information from the trained tree
    booster = model.get_booster()
    tree_df = booster.trees_to_dataframe() # May error depending on the model structure.

    # Extract split values corresponding to tenure_col
    tenure_splits = tree_df[tree_df['Feature'] == tenure_col]['Split'].dropna().unique()
    tenure_splits.sort()

    min_val = df_processed[tenure_col].min() # Error if df_processed is empty
    max_val = df_processed[tenure_col].max()

    if len(tenure_splits) == 0:
        # If there are no splits for tenure_col, return the entire range as a single bin.
        return [min_val, max_val + 1e-9 if min_val == max_val else max_val]

    # Create the final bin edges
    bin_edges = sorted(list(set([min_val] + list(tenure_splits) + [max_val])))

    # Check and adjust bin edge validity (for pd.cut)
    if len(bin_edges) < 2:
        return [min_val, max_val + 1e-9 if min_val == max_val else max_val]
    if len(bin_edges) == 2 and bin_edges[0] == bin_edges[1]: # If all tenure values are the same
        bin_edges = [bin_edges[0], bin_edges[1] + 1e-9]

    return bin_edges

## 3,4로 할지? 몇개 정도로 하면 좋을지 논의

In [ ]:
# 1. Find optimal tenure bins using XGBoost
#    (An error will occur here if df is missing, column names are incorrect, or the data is unsuitable.)
tenure_bin_edges_xgb = find_optimal_tenure_bins_xgb(
    df_input=df, 
    tenure_col='tenure',
    target_col='forced_3',
    max_tree_depth=3,  # Example: max tree depth of 3
    random_state=42    # random_state for reproducibility
)
print(f"Determined Tenure bin edges (XGBoost-based, error handling removed): {tenure_bin_edges_xgb}")

# 2. Add the new binned tenure column and check the results
#    (Assuming tenure_bin_edges_xgb is returned in a format suitable for pd.cut)
'''
df['tenure_binned_xgb'] = pd.cut(
    df['tenure'],
    bins=tenure_bin_edges_xgb, # This list must have at least two elements for pd.cut to work correctly
    right=True,
    include_lowest=True,
    duplicates='drop' # Handles duplicate bin edges automatically (a safeguard in pd.cut)
)

print("\nFrequency count per Tenure bin (XGBoost-based, error handling removed):")
print(df['tenure_binned_xgb'].value_counts().sort_index())

# 3. (Optional) Create dummy variables
#tenure_dummies_xgb = pd.get_dummies(df['tenure_binned_xgb'], prefix='tenure_bin_xgb_s', dummy_na=False)
#print("\nTenure dummy variables (XGBoost-based, error handling removed, top 5 rows):")
#print(tenure_dummies_xgb.head())
#print("\n'tenure_dummies_xgb' can now be used as features in a model.")
'''

결정된 Tenure 구간 경계 (XGBoost 기반, 오류 처리 제거): [np.int64(1), np.float64(2.0), np.float64(7.0), np.float64(10.0), np.float64(17.0), np.float64(21.0), np.float64(46.0), np.int64(62)]

Tenure 구간별 빈도수 (XGBoost 기반, 오류 처리 제거):
tenure_binned_xgb
(0.999, 2.0]     8074
(2.0, 7.0]      15866
(7.0, 10.0]      5248
(10.0, 17.0]     5617
(17.0, 21.0]     1258
(21.0, 46.0]     1839
(46.0, 62.0]       43
Name: count, dtype: int64


/root/miniconda3/envs/deep_env/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [14:12:58] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


(0.999, 2.0]     8074  
(2.0, 10.0]     21114  
(10.0, 21.0]     6875  
(21.0, 62.0]     1882  
  
(0.999, 2.0]     8074  
(2.0, 7.0]      15866  
(7.0, 10.0]      5248  
(10.0, 17.0]     5617  
(17.0, 21.0]     1258  
(21.0, 46.0]     1839  
(46.0, 62.0]       43  
  
depth3 is more uniformly distributed, so use depth3 to make tenure indicator

In [ ]:
def create_tenure_indicator_columns(df, tenure_column_name, bin_edges):
    df_copy = df.copy() 
    
    if not isinstance(bin_edges, list) or len(bin_edges) < 2:
        raise ValueError("bin_edges must be a list with at least 2 values.")

    # Create an indicator column for each interval
    for i in range(len(bin_edges) - 1):
        lower_bound = bin_edges[i]
        upper_bound = bin_edges[i+1]
        
        new_col_name = f'I{tenure_column_name}{i+1}' # Set the new column name format, e.g., Itenure1
        
        condition = False # Initialize condition
        if i == 0:  # For the first indicator column
            # Change condition to be inclusive of the lower_bound (tenure >= lower_bound)
            condition = (df_copy[tenure_column_name] >= lower_bound) & \
                        (df_copy[tenure_column_name] <= upper_bound)
        else:  # For all other indicator columns
            # Keep the original condition (tenure > lower_bound)
            condition = (df_copy[tenure_column_name] > lower_bound) & \
                        (df_copy[tenure_column_name] <= upper_bound)
            
        df_copy[new_col_name] = condition.astype(int)
        
    return df_copy

In [ ]:
bin_edges_list = tenure_bin_edges_xgb
df = create_tenure_indicator_columns(df, 'tenure', bin_edges_list)

In [ ]:
bin_edges_list

[np.int64(1),
 np.float64(2.0),
 np.float64(10.0),
 np.float64(21.0),
 np.int64(62)]

# traintest flag
3 sets each for (forced:not forced) 1:1 and 1:2 ratios, using forced_3 and forced_3_4 as target variable

In [ ]:
def matched_sampling(df: pd.DataFrame, target_col: str, ratio: int, 
                                               test_ratio: float = 0.2, 
                                               random_state: int = None) -> pd.DataFrame:
    
    """
    Performs matched case-control sampling and adds train/test indicator columns.

    Args:
        df (pd.DataFrame): The input DataFrame.
        target_col (str): The name of the classification target column.
        ratio (int): The sampling ratio for train:test, expressed as 'ratio:1'. 
                    (e.g., ratio=1 for a 1:1 split, ratio=2 for a 2:1 split).
        test_ratio (float, optional): The proportion of the entire dataset to allocate to the test set. 
                                    Defaults to 0.2.

    Returns:
        pd.DataFrame: The original DataFrame with 'train' and 'test' columns added.
    """
    df_copy = df.copy()

    # 1. Case (target_col == 1)와 Control (target_col == 0) 분리
    df_case = df_copy[df_copy[target_col] == 1]
    df_control = df_copy[df_copy[target_col] == 0]

    n_case = len(df_case)
    n_control_needed = n_case * ratio

    if n_control_needed > len(df_control):
        print(f"Warning: 요청된 control 샘플 수({n_control_needed})가 실제 control 샘플 수({len(df_control)})보다 많습니다. 가능한 모든 control 샘플을 사용합니다.")
        n_control_to_sample = len(df_control)
    else:
        n_control_to_sample = n_control_needed
        
    # 2. Control 샘플링
    df_control_sampled = df_control.sample(n=n_control_to_sample, random_state=random_state)

    # 3. 샘플링된 데이터 합치기
    df_sampled = pd.concat([df_case, df_control_sampled])

    # 4. Train/Test 분리 (샘플링된 데이터 내에서)
    if not df_sampled.empty:
        # 인덱스를 기준으로 분리하기 위해 인덱스만 사용
        sampled_indices = df_sampled.index
        
        # target_col을 기준으로 계층적 분할(stratify)을 고려할 수 있으나,
        # case-control 샘플링으로 이미 비율이 조절되었으므로 단순 분할도 가능합니다.
        # 여기서는 df_sampled[target_col]을 사용하여 계층적 분할을 시도합니다.
        # 만약 df_sampled가 매우 작아서 계층화가 불가능한 경우를 대비해 예외처리
        try:
            train_indices, test_indices = train_test_split(
                sampled_indices,
                test_size=test_ratio,
                random_state=random_state,
                stratify=df_sampled[target_col] if len(df_sampled[target_col].unique()) > 1 else None
            )
        except ValueError: # stratify가 불가능한 경우 (예: 한 클래스만 있거나 샘플이 너무 적을 때)
             train_indices, test_indices = train_test_split(
                sampled_indices,
                test_size=test_ratio,
                random_state=random_state
            )

    else: # 샘플링된 데이터가 없는 경우 (예: case가 없는 경우)
        train_indices = pd.Index([])
        test_indices = pd.Index([])

    # 5. 'train'과 'test' 컬럼 초기화 (모두 0으로)
    df_copy['train'] = 0
    df_copy['test'] = 0

    # 6. 샘플링된 데이터에 따라 'train', 'test' 컬럼 값 업데이트
    df_copy.loc[train_indices, 'train'] = 1
    df_copy.loc[test_indices, 'test'] = 1
    
    return df_copy

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def matched_sampling_multiple(df: pd.DataFrame, 
                                   target_col: str, 
                                   ratio: int, 
                                   num_samplings: int, # Specifies how many times to repeat
                                   test_ratio: float = 0.2, 
                                   random_state: int = None) -> pd.DataFrame:
    """
    A function that performs matched case-control sampling and train/test splitting
    multiple times for an imbalanced target column, and adds train/test columns for each iteration.

    Args:
        df (pd.DataFrame): The input DataFrame.
        target_col (str): The name of the classification target column.
        ratio (int): The sampling ratio of (count of target_col == 0) / (count of target_col == 1).
                     (e.g., ratio=2 means sampling at 1:2)
        num_samplings (int): The number of times to repeat the sampling and splitting.
        test_ratio (float, optional): The proportion of the test set from the total sampled data. 
                                      Defaults to 0.2.
        random_state (int, optional): random_state for reproducibility. 
                                      Each iteration gets a different random_state based on this value. Defaults to None.

    Returns:
        pd.DataFrame: The original DataFrame with 'trainX' and 'testX' columns added for each iteration.
    """
    df_copy = df.copy()

    # 1. Separate Case (target_col == 1) and Control (target_col == 0) (performed only once)
    df_case = df_copy[df_copy[target_col] == 1]
    df_control = df_copy[df_copy[target_col] == 0]

    n_case = len(df_case)
    
    if n_case == 0:
        print("Warning: No Case samples (target_col == 1) found. Cannot proceed with sampling.")
        # Initialize all requested train/test columns to 0
        for i in range(1, num_samplings + 1):
            df_copy[f'train{i}'] = 0
            df_copy[f'test{i}'] = 0
        return df_copy

    n_control_needed = n_case * ratio
    actual_n_control = len(df_control)

    if n_control_needed > actual_n_control:
        print(f"Warning: For all iterations, the requested number of control samples ({n_control_needed}) is greater than the available number of control samples ({actual_n_control}). Using all available control samples.")
        n_control_to_sample_for_each_iteration = actual_n_control
    else:
        n_control_to_sample_for_each_iteration = n_control_needed
    
    if ratio > 0 and actual_n_control == 0:
        print("Warning: No Control samples (target_col == 0) exist, but ratio is greater than 0. Splitting will proceed with Case samples only.")
        n_control_to_sample_for_each_iteration = 0


    # Repeat sampling and splitting multiple times
    for i in range(1, num_samplings + 1):
        current_iter_random_state = None
        if random_state is not None:
            # Use a different random_state for each iteration, while keeping the entire process reproducible
            current_iter_random_state = random_state + (i - 1) 

        # 2. Sample Controls (newly sampled from the original control pool in each iteration)
        if n_control_to_sample_for_each_iteration > 0 and not df_control.empty:
            df_control_sampled = df_control.sample(n=n_control_to_sample_for_each_iteration, 
                                                   random_state=current_iter_random_state)
            # 3. Combine the sampled data
            df_sampled_iter = pd.concat([df_case, df_control_sampled])
        elif not df_case.empty: # If control sampling is not needed (ratio=0) or not possible (no controls), use Case samples only
            df_sampled_iter = df_case.copy()
            if n_control_needed > 0 and df_control.empty and i==1: # This warning is printed only once
                 print(f"No Control samples found. Proceeding with train/test split using Case samples only.")
        else: # The case of no Cases is already handled above
            df_sampled_iter = pd.DataFrame()


        # Generate train/test column names for the current iteration
        train_col_name = f'train{i}'
        test_col_name = f'test{i}'
        
        # Initialize columns
        df_copy[train_col_name] = 0
        df_copy[test_col_name] = 0

        if not df_sampled_iter.empty:
            sampled_indices_iter = df_sampled_iter.index
            
            stratify_on = None
            if len(df_sampled_iter[target_col].unique()) > 1 and len(df_sampled_iter) >= 2 : # Stratification requires at least 2 samples and 2 classes
                 # Also need to consider if each class has enough samples for the split size (handled internally by train_test_split)
                try:
                    # Temporarily check value_counts to more reliably determine if stratification is possible
                    vc = df_sampled_iter[target_col].value_counts()
                    if all(c >= 2 for c in vc): # At least 2 classes, with at least 2 samples per class (more strictly related to n_splits)
                         stratify_on = df_sampled_iter[target_col]
                    # else: stratify_on remains None
                except: # General exception handling for complex situations
                    pass # stratify_on remains None

            try:
                train_indices, test_indices = train_test_split(
                    sampled_indices_iter,
                    test_size=test_ratio,
                    random_state=current_iter_random_state,
                    stratify=stratify_on
                )
            except ValueError: # If stratification is not possible (e.g., only one class or too few samples)
                 train_indices, test_indices = train_test_split(
                    sampled_indices_iter,
                    test_size=test_ratio,
                    random_state=current_iter_random_state
                )
            
            # Update column values
            df_copy.loc[train_indices, train_col_name] = 1
            df_copy.loc[test_indices, test_col_name] = 1
        else:
            # This case occurs when df_case is empty and is already handled at the beginning of the function.
            # Theoretically, this part should not be reached.
            print(f"Iteration {i}: No data was sampled, so columns {train_col_name} and {test_col_name} cannot be populated.")
            
    return df_copy

In [ ]:
df_result12 = matched_sampling_multiple(df, target_col='forced_3', ratio = 2, num_samplings=3, test_ratio=0.2, random_state=42)
df_result12.to_csv('data/matched_sampling_1:2.csv', index=False)
df_result11 = matched_sampling_multiple(df, target_col='forced_3', ratio = 1, num_samplings=3, test_ratio=0.2, random_state=42)
df_result11.to_csv('data/matched_sampling_1:1.csv', index=False)